## OpenEO Basics

Installation, importation and making a connection to a backend

In [ ]:
!pip install openeo

In [2]:
import openeo

print(openeo.client_version())

0.9.1


In [3]:
connection = openeo.connect("https://openeo.cloud").authenticate_oidc()

To authenticate: visit https://aai.egi.eu/oidc/device and enter the user code 'JZK4L2'.
Authorized successfully.
Authenticated using device code flow.


Loading a collection

In [5]:
s2_cube = connection.load_collection(
    "TERRASCOPE_S2_TOC_V2",
    spatial_extent={"west":5.1,"east":5.2,"south":51.1,"north":51.2},
    temporal_extent=["2020-01-01","2020-12-31"],
    bands=["B02","B04","B08"]
)

Downloading results: synchronous job

In [8]:
s2_cube.max_time().download("results/basics/s2_cube_syn.tif",format="Gtiff")

## Note: we used max_time() so that you only need to download 1 time interval just to speed up the download process
## for this particular demo. To see what the method does and to see more methods of a DataCube, you can check out:
## https://open-eo.github.io/openeo-python-client/api.html#openeo.rest.datacube.DataCube.max_time

Downloading results: batch job

In [12]:
job = s2_cube.max_time().execute_batch(format="Gtiff")
job.get_results().download_file("results/basics/s2_cube_batch.tif")

0:00:00 Job 'vito-305a8796-63c6-40e1-add2-8a8f82f51c43': send 'start'
0:00:55 Job 'vito-305a8796-63c6-40e1-add2-8a8f82f51c43': queued (progress N/A)
0:01:03 Job 'vito-305a8796-63c6-40e1-add2-8a8f82f51c43': queued (progress N/A)
0:01:10 Job 'vito-305a8796-63c6-40e1-add2-8a8f82f51c43': queued (progress N/A)
0:01:19 Job 'vito-305a8796-63c6-40e1-add2-8a8f82f51c43': queued (progress N/A)
0:01:30 Job 'vito-305a8796-63c6-40e1-add2-8a8f82f51c43': queued (progress N/A)
0:01:44 Job 'vito-305a8796-63c6-40e1-add2-8a8f82f51c43': queued (progress N/A)
0:02:01 Job 'vito-305a8796-63c6-40e1-add2-8a8f82f51c43': running (progress N/A)
0:02:23 Job 'vito-305a8796-63c6-40e1-add2-8a8f82f51c43': running (progress N/A)
0:02:48 Job 'vito-305a8796-63c6-40e1-add2-8a8f82f51c43': running (progress N/A)
0:03:20 Job 'vito-305a8796-63c6-40e1-add2-8a8f82f51c43': running (progress N/A)
0:03:59 Job 'vito-305a8796-63c6-40e1-add2-8a8f82f51c43': finished (progress N/A)


WindowsPath('results/basics/s2_cube_batch.tif')

Filtering spatially, temporally and on bands

In [23]:
b2_filtered_cropped = s2_cube.filter_temporal(["2020-02-01","2020-02-28"]).band("B02").filter_bbox(west=5.15,east=5.16,south=51.14,north=51.16, crs=4326)
# job = b2_filtered_cropped.execute_batch(format="netCDF")
job = b2_filtered_cropped.send_job(out_format="netCDF")
job.start_job()
print(job.job_id)
print(job.status())
# job.get_results().download_file("results/basics/s2_filtered_cropped.nc")
# b2_filtered_cropped.download("results/basics/b2_filtered_cropped.nc", format="netCDF")

vito-d4f4b476-614e-47d5-b8a7-6544fca3835f
queued


In [24]:
print(job.status())

running


Band math

In [26]:
B02 = s2_cube.band("B02")
B04 = s2_cube.band("B04")
B08 = s2_cube.band("B08")
ndvi_cube = (B08 - B04) / (B08 + B04)
ndvi_cube.max_time().download("results/basics/ndvi.tif",format="GTiff")

Applying mathematical operations

In [ ]:
s2_absolute = s2_cube.apply("absolute")
s2_lambda = s2_cube.apply(lambda x: x.absolute().cos())

In [45]:
s2_red = s2_cube.reduce_dimension(dimension="t",reducer="max")

In [31]:
from openeo.processes import array_element
def callback(data):
    band1 = array_element(data, index=0)
    band2 = array_element(data, index=1)
    return band1 + 1.2 * band2

s2_cube.reduce_dimension(dimension="bands",reducer=callback)

In [ ]:
def callback2(data):
    return data.mean()

s2_cube.reduce_dimension(dimension="t",reducer=callback2)

Sampling features

In [ ]:
with open("resources/soy_2019.geojson") as fn:
    pol = fn.readlines()[0]

sampled_features = ndvi_cube.filter_spatial(json.loads(pol))
job = sampled_features.send_job(
    title="Extract points from SHub",
    out_format="netCDF",
    sample_by_feature=True)
job.start_job()

In [52]:
job.status()

'finished'